<a href="https://colab.research.google.com/github/Antoine723/ProjetNLP/blob/main/ProjetNLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [17]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.text import Text
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem import WordNetLemmatizer
import numpy as np
import tensorflow as tf

from gdrive.MyDrive.ProjetNLP.rnn_utils import *

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [18]:
df=pd.read_csv('gdrive/MyDrive/ProjetNLP/training.1600000.processed.noemoticon.csv',encoding="ISO-8859-1",header=None)
data=np.array([df.iloc[0:5][5],df.iloc[800000:800005][5]])

print(data)

[["@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D"
  "is upset that he can't update his Facebook by texting it... and might cry as a result  School today also. Blah!"
  '@Kenichan I dived many times for the ball. Managed to save 50%  The rest go out of bounds'
  'my whole body feels itchy and like its on fire '
  "@nationwideclass no, it's not behaving at all. i'm mad. why am i here? because I can't see you all over there. "]
 ['I LOVE @Health4UandPets u guys r the best!! '
  'im meeting up with one of my besties tonight! Cant wait!!  - GIRL TALK!!'
  '@DaRealSunisaKim Thanks for the Twitter add, Sunisa! I got to meet you once at a HIN show here in the DC area and you were a sweetheart. '
  'Being sick can be really cheap when it hurts too much to eat real food  Plus, your friends make you soup'
  '@LovesBrooklyn2 he has that effect on everyone ']]


# Traitement

In [19]:
def processing(data):

  #MINUSCULES
  for i in range(len(data)):
    for j in range(len(data[i])):
      data[i][j]=data[i][j].lower()
  # print(data)

  #TOKENIZATION
  tokenizer = nltk.RegexpTokenizer(r'\w+')

  for i in range(len(data)):
    for j in range(len(data[i])):
      data[i][j]=tokenizer.tokenize(data[i][j])
  # print(data)

  #STOPWORDS
  for i in range(len(data)):
    for j in range(len(data[i])):
      data[i][j]=[w for w in data[i][j] if not w in list(nltk.corpus.stopwords.words('english'))]
      
  # print(data)

  #STEMMING
  stemmer=LancasterStemmer()
  for i in range(len(data)):
    for j in range(len(data[i])):
      data[i][j]=[stemmer.stem(w) for w in data[i][j]]
  # print(data)

  #LEMMATIZATION
  Word_Lemmatizer = WordNetLemmatizer()
  for i in range(len(data)):
    for j in range(len(data[i])):
      data[i][j]=[Word_Lemmatizer.lemmatize(w) for w in data[i][j]]
  print(data)

  return data

In [20]:
datas=processing(data)

[[list(['switchfoot', 'http', 'twitp', 'com', '2y1zl', 'awww', 'bum', 'should', 'got', 'david', 'car', 'third', 'day'])
  list(['upset', 'upd', 'facebook', 'text', 'might', 'cry', 'result', 'school', 'today', 'also', 'blah'])
  list(['kenich', 'div', 'many', 'tim', 'bal', 'man', 'sav', '50', 'rest', 'go', 'bound'])
  list(['whol', 'body', 'feel', 'itchy', 'lik', 'fir'])
  list(['nationwideclass', 'behav', 'mad', 'see'])]
 [list(['lov', 'health4uandpets', 'u', 'guy', 'r', 'best'])
  list(['im', 'meet', 'on', 'besty', 'tonight', 'cant', 'wait', 'girl', 'talk'])
  list(['darealsunisakim', 'thank', 'twit', 'ad', 'sunis', 'got', 'meet', 'hin', 'show', 'dc', 'are', 'sweetheart'])
  list(['sick', 'real', 'cheap', 'hurt', 'much', 'eat', 'real', 'food', 'plu', 'friend', 'mak', 'soup'])
  list(['lovesbrooklyn2', 'effect', 'everyon'])]]


In [21]:
from collections import defaultdict
v=set()
for i in range(len(data)):
  for j in range(len(data[i])):
    for h in range(len(data[i][j])):
      v.add(data[i][j][h])

vocab = defaultdict(lambda: 0)
vocab['<PAD>'] = 1

j=2
for i in v:
  vocab[i]=j
  j+=1
print(vocab)

defaultdict(<function <lambda> at 0x7f6c5e53d400>, {'<PAD>': 1, 'im': 2, 'kenich': 3, 'sweetheart': 4, 'result': 5, 'feel': 6, 'show': 7, 'sav': 8, 'whol': 9, 'hurt': 10, 'might': 11, 'girl': 12, '2y1zl': 13, 'switchfoot': 14, 'bum': 15, 'mad': 16, 'meet': 17, 'u': 18, 'ad': 19, 'david': 20, 'third': 21, 'twit': 22, 'div': 23, 'com': 24, 'on': 25, '50': 26, 'go': 27, 'upd': 28, 'food': 29, 'mak': 30, 'fir': 31, 'body': 32, 'health4uandpets': 33, 'darealsunisakim': 34, 'blah': 35, 'tim': 36, 'also': 37, 'facebook': 38, 'lik': 39, 'behav': 40, 'man': 41, 'cant': 42, 'see': 43, 'day': 44, 'best': 45, 'sick': 46, 'twitp': 47, 'real': 48, 'should': 49, 'cheap': 50, 'soup': 51, 'effect': 52, 'everyon': 53, 'dc': 54, 'eat': 55, 'upset': 56, 'text': 57, 'itchy': 58, 'thank': 59, 'hin': 60, 'many': 61, 'lov': 62, 'plu': 63, 'sunis': 64, 'awww': 65, 'tonight': 66, 'friend': 67, 'r': 68, 'much': 69, 'cry': 70, 'are': 71, 'lovesbrooklyn2': 72, 'talk': 73, 'besty': 74, 'bal': 75, 'school': 76, 'got

In [22]:
def transformWords(data):
  matrixData=[]
  maxLen=0
  for i in range(len(data)):
    for j in range(len(data[i])):
      if (maxLen < len(data[i][j])):
        maxLen= len(data[i][j])

  for i in range(len(data)):
    for j in range(len(data[i])):
      test=[]
      for k in range(maxLen):
        if (k >= len(data[i][j])):
          test.append(vocab['<PAD>'])
        else:
          test.append(vocab[data[i][j][k]])
      matrixData.append(test)
  return matrixData

In [23]:
data=transformWords(data)
datas=np.array([df.iloc[5:10][5],df.iloc[800005:800010][5]])
datas=processing(datas)
datas=transformWords(datas)

[[list(['kweside', 'whol', 'crew']) list(['nee', 'hug'])
  list(['lolt', 'hey', 'long', 'tim', 'see', 'ye', 'rain', 'bit', 'bit', 'lol', 'fin', 'thank'])
  list(['tatiana_k', 'nop']) list(['twitter', 'que', 'muer'])]
 [list(['productoffear', 'tel', 'burst', 'laugh', 'real', 'loud', 'thank', 'mak', 'com', 'sulk'])
  list(['r_keith_hill', 'than', 'respons', 'ihad', 'already', 'find', 'answ'])
  list(['keepinupwkr', 'jeal', 'hop', 'gre', 'tim', 'vega', 'lik', 'acm', 'lov', 'show'])
  list(['tommcf', 'ah', 'congr', 'mr', 'fletch', 'fin', 'join', 'twit'])
  list(['e4voip', 'respond', 'stupid', 'cat', 'help', 'typ', 'forg', 'er'])]]


In [62]:
model=tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(input_dim=len(vocab),output_dim=2))
model.add(tf.keras.layers.LSTM(128,activation="relu"))
model.add(tf.keras.layers.Dense(1,activation="softmax"))

In [25]:
data=np.array(data)

In [26]:
data

array([[14, 80, 47, 24, 13, 65, 15, 49, 77, 20, 78, 21, 44],
       [56, 28, 38, 57, 11, 70,  5, 76, 79, 37, 35,  1,  1],
       [ 3, 23, 61, 36, 75, 41,  8, 26, 83, 27, 82,  1,  1],
       [ 9, 32,  6, 58, 39, 31,  1,  1,  1,  1,  1,  1,  1],
       [85, 40, 16, 43,  1,  1,  1,  1,  1,  1,  1,  1,  1],
       [62, 33, 18, 84, 68, 45,  1,  1,  1,  1,  1,  1,  1],
       [ 2, 17, 25, 74, 66, 42, 81, 12, 73,  1,  1,  1,  1],
       [34, 59, 22, 19, 64, 77, 17, 60,  7, 54, 71,  4,  1],
       [46, 48, 50, 10, 69, 55, 48, 29, 63, 67, 30, 51,  1],
       [72, 52, 53,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1]])

In [27]:
datas=np.array(datas)

In [63]:
print(data)
print(datas)

[[14 80 47 24 13 65 15 49 77 20 78 21 44]
 [56 28 38 57 11 70  5 76 79 37 35  1  1]
 [ 3 23 61 36 75 41  8 26 83 27 82  1  1]
 [ 9 32  6 58 39 31  1  1  1  1  1  1  1]
 [85 40 16 43  1  1  1  1  1  1  1  1  1]
 [62 33 18 84 68 45  1  1  1  1  1  1  1]
 [ 2 17 25 74 66 42 81 12 73  1  1  1  1]
 [34 59 22 19 64 77 17 60  7 54 71  4  1]
 [46 48 50 10 69 55 48 29 63 67 30 51  1]
 [72 52 53  1  1  1  1  1  1  1  1  1  1]]
[[ 0  9  0  1  1  1  1  1  1  1  1  1]
 [ 0  0  1  1  1  1  1  1  1  1  1  1]
 [ 0  0  0 36 43  0  0  0  0  0  0 59]
 [ 0  0  1  1  1  1  1  1  1  1  1  1]
 [ 0  0  0  1  1  1  1  1  1  1  1  1]
 [ 0  0  0  0 48  0 59 30 24  0  1  1]
 [ 0  0  0  0  0  0  0  1  1  1  1  1]
 [ 0  0  0  0 36  0 39  0 62  7  1  1]
 [ 0  0  0  0  0  0  0 22  1  1  1  1]
 [ 0  0  0  0  0  0  0  0  1  1  1  1]]


In [64]:
y_train=np.concatenate([np.zeros((5,1)),np.ones((5,1))])
model.compile(loss='binary_crossentropy',optimizer="sgd")
model.fit(data,y_train,validation_data=(datas,y_train),epochs=50)

Epoch 1/50
1/1 [==============================] - 2s 2s/step - loss: 0.6931 - val_loss: 0.6931
Epoch 2/50
1/1 [==============================] - 0s 70ms/step - loss: 0.6931 - val_loss: 0.6931
Epoch 3/50
1/1 [==============================] - 0s 62ms/step - loss: 0.6930 - val_loss: 0.6931
Epoch 4/50
1/1 [==============================] - 0s 71ms/step - loss: 0.6930 - val_loss: 0.6931
Epoch 5/50
1/1 [==============================] - 0s 66ms/step - loss: 0.6930 - val_loss: 0.6931
Epoch 6/50
1/1 [==============================] - 0s 57ms/step - loss: 0.6930 - val_loss: 0.6931
Epoch 7/50
1/1 [==============================] - 0s 65ms/step - loss: 0.6930 - val_loss: 0.6931
Epoch 8/50
1/1 [==============================] - 0s 72ms/step - loss: 0.6930 - val_loss: 0.6932
Epoch 9/50
1/1 [==============================] - 0s 72ms/step - loss: 0.6930 - val_loss: 0.6932
Epoch 10/50
1/1 [==============================] - 0s 65ms/step - loss: 0.6930 - val_loss: 0.6932
Epoch 11/50
1/1 [==============

In [65]:
model.predict(data[0])

array([[1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.]], dtype=float32)